In [1123]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import sklearnimport time

In [1124]:
df = pd.read_csv('data.csv')
df = df[-400:]
df

,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48
623,Hakeem Olajuwon,33,HOU,1.0,238.0,1130,0.211,72,38.8,26.9,10.9,3.6,1.6,2.9,0.514,0.214,0.724,9.7,0.166
624,Scottie Pippen,30,CHI,0.0,226.0,1130,0.200,77,36.7,19.4,6.4,5.9,1.7,0.7,0.463,0.374,0.679,12.3,0.209
625,Gary Payton,27,SEA,0.0,98.0,1130,0.087,81,39.0,19.3,4.2,7.5,2.9,0.2,0.484,0.328,0.748,11.5,0.174
626,Karl Malone,32,UTA,1.0,85.0,1130,0.075,82,38.0,25.7,9.8,4.2,1.7,0.7,0.519,0.400,0.723,15.1,0.233
627,Shawn Kemp,26,SEA,0.0,73.0,1130,0.065,79,33.3,19.6,11.4,2.2,1.2,1.6,0.561,0.417,0.742,11.2,0.205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,Stephen Curry,33,GSW,0.0,4.0,1000,0.004,64,34.5,25.5,5.2,6.3,1.3,0.4,0.437,0.380,0.923,8.0,0.173
1019,Chris Paul,36,PHO,0.0,2.0,1000,0.002,65,32.9,14.7,4.4,10.8,1.9,0.3,0.493,0.317,0.837,9.4,0.210
1020,DeMar DeRozan,32,CHI,0.0,1.0,1000,0.001,76,36.1,27.9,5.2,4.9,0.9,0.3,0.504,0.352,0.877,8.8,0.154
1021,Kevin Durant,33,BRK,0.0,1.0,1000,0.001,55,37.2,29.9,7.4,6.4,0.9,0.9,0.518,0.383,0.910,8.4,0.198


In [1125]:
cur_stats = pd.read_csv('current_stats.csv')
cur_stats_adv = pd.read_csv('current_stats_adv.csv')

cur_stats['WS'] = cur_stats_adv['WS']
cur_stats['WS/48'] = cur_stats_adv['WS/48']
cur_stats = cur_stats.drop(['Tm'], axis=1)

cur_stats = cur_stats.drop([col for col in cur_stats.columns if col not in df.columns], axis=1)
cur_stats.head()

,Player,Age,G,MP,FG%,3P%,FT%,TRB,AST,STL,BLK,PTS,WS,WS/48
0,Precious Achiuwa,23,34,23.3,0.486,0.247,0.684,6.5,1.1,0.7,0.7,10.4,1.6,0.099
1,Steven Adams,29,42,27.0,0.597,0.000,0.364,11.5,2.3,0.9,1.1,8.6,3.4,0.145
2,Bam Adebayo,25,52,35.3,0.546,0.091,0.806,10.1,3.3,1.2,0.8,21.6,6.1,0.159
3,Ochai Agbaji,22,35,14.0,0.486,0.394,0.625,1.6,0.5,0.1,0.1,4.1,0.7,0.068
4,Santi Aldama,22,53,22.1,0.477,0.367,0.738,4.7,1.2,0.7,0.7,9.5,3.6,0.146


In [1126]:
# Remove Player and Team and drop duplicate
df = df.drop(['Player', 'Tm'], axis=1)
df.head()

,Age,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48
623,33,1.0,238.0,1130,0.211,72,38.8,26.9,10.9,3.6,1.6,2.9,0.514,0.214,0.724,9.7,0.166
624,30,0.0,226.0,1130,0.200,77,36.7,19.4,6.4,5.9,1.7,0.7,0.463,0.374,0.679,12.3,0.209
625,27,0.0,98.0,1130,0.087,81,39.0,19.3,4.2,7.5,2.9,0.2,0.484,0.328,0.748,11.5,0.174
626,32,1.0,85.0,1130,0.075,82,38.0,25.7,9.8,4.2,1.7,0.7,0.519,0.400,0.723,15.1,0.233
627,26,0.0,73.0,1130,0.065,79,33.3,19.6,11.4,2.2,1.2,1.6,0.561,0.417,0.742,11.2,0.205


In [1127]:
# Setting NaN features to the average

# Calculating average
df_mean = df.dropna()
mean_vals = df_mean.mean()

# Setting values
for row in range(len(df)):
    for i, val in enumerate(df.iloc[row]):
        if np.isnan(val):
            df.iat[row,i] = mean_vals[i]

df.head()

,Age,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48
623,33,1.0,238.0,1130,0.211,72,38.8,26.9,10.9,3.6,1.6,2.9,0.514,0.214,0.724,9.7,0.166
624,30,0.0,226.0,1130,0.200,77,36.7,19.4,6.4,5.9,1.7,0.7,0.463,0.374,0.679,12.3,0.209
625,27,0.0,98.0,1130,0.087,81,39.0,19.3,4.2,7.5,2.9,0.2,0.484,0.328,0.748,11.5,0.174
626,32,1.0,85.0,1130,0.075,82,38.0,25.7,9.8,4.2,1.7,0.7,0.519,0.400,0.723,15.1,0.233
627,26,0.0,73.0,1130,0.065,79,33.3,19.6,11.4,2.2,1.2,1.6,0.561,0.417,0.742,11.2,0.205


In [1128]:
# Dropping First, Pts Won, and Pts Max. Using MVP shares for as the predicted value
df = df.drop(['First', 'Pts Won', 'Pts Max'], axis=1)
df.tail()

,Age,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48
1018,33,0.004,64,34.5,25.5,5.2,6.3,1.3,0.4,0.437,0.380,0.923,8.0,0.173
1019,36,0.002,65,32.9,14.7,4.4,10.8,1.9,0.3,0.493,0.317,0.837,9.4,0.210
1020,32,0.001,76,36.1,27.9,5.2,4.9,0.9,0.3,0.504,0.352,0.877,8.8,0.154
1021,33,0.001,55,37.2,29.9,7.4,6.4,0.9,0.9,0.518,0.383,0.910,8.4,0.198
1022,37,0.001,56,37.2,30.3,8.2,6.2,1.3,1.1,0.524,0.359,0.756,7.5,0.172


In [1129]:
full_df = pd.concat([cur_stats, df])
normalized_df = full_df.drop(['Player', 'Share'], axis=1)
normalized_df['WS'] = list(map(lambda x: x**2, normalized_df['WS']))

In [1130]:
normalized_df = (normalized_df-normalized_df.min())/(normalized_df.max()-normalized_df.min())
normalized_df['Player'] = full_df['Player']
normalized_df['Share'] = full_df['Share']
normalized_df

,Age,G,MP,FG%,3P%,FT%,TRB,AST,STL,BLK,PTS,WS,WS/48,Player,Share
0,0.173913,0.407407,0.533181,0.486,0.247,0.684,0.422078,0.094017,0.175,0.179487,0.288089,0.006212,0.458207,Precious Achiuwa,NaN
1,0.434783,0.506173,0.617849,0.597,0.000,0.364,0.746753,0.196581,0.225,0.282051,0.238227,0.028052,0.481370,Steven Adams,NaN
2,0.260870,0.629630,0.807780,0.546,0.091,0.806,0.655844,0.282051,0.300,0.205128,0.598338,0.090296,0.488419,Bam Adebayo,NaN
3,0.130435,0.419753,0.320366,0.486,0.394,0.625,0.103896,0.042735,0.025,0.025641,0.113573,0.001189,0.442598,Ochai Agbaji,NaN
4,0.130435,0.641975,0.505721,0.477,0.367,0.738,0.305195,0.102564,0.175,0.179487,0.263158,0.031449,0.481873,Santi Aldama,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,0.608696,0.777778,0.789474,0.437,0.380,0.923,0.337662,0.538462,0.325,0.102564,0.706371,0.155306,0.495468,NaN,0.004
1019,0.739130,0.790123,0.752860,0.493,0.317,0.837,0.285714,0.923077,0.475,0.076923,0.407202,0.214419,0.514099,NaN,0.002
1020,0.565217,0.925926,0.826087,0.504,0.352,0.877,0.337662,0.418803,0.225,0.076923,0.772853,0.187920,0.485901,NaN,0.001
1021,0.608696,0.666667,0.851259,0.518,0.383,0.910,0.480519,0.547009,0.225,0.230769,0.828255,0.171225,0.508056,NaN,0.001


In [1131]:
# normalized_df = normalized_df[['WS','WS/48', 'Player', 'Share']]

In [1132]:
data = normalized_df.loc[normalized_df['Player'].isna()].drop(['Player'], axis=1).to_numpy()
y = data[:, -1]
data = np.delete(data, -1, axis=1)
X = data
X, y

(array([[0.60869565, 0.87654321, 0.88787185, ..., 0.74515235, 0.22832391,
         0.49194361],
        [0.47826087, 0.9382716 , 0.83981693, ..., 0.53739612, 0.36712854,
         0.51359517],
        [0.34782609, 0.98765432, 0.89244851, ..., 0.53462604, 0.32092504,
         0.4959718 ],
        ...,
        [0.56521739, 0.92592593, 0.82608696, ..., 0.77285319, 0.18792011,
         0.48590131],
        [0.60869565, 0.66666667, 0.85125858, ..., 0.82825485, 0.17122473,
         0.50805639],
        [0.7826087 , 0.67901235, 0.85125858, ..., 0.83933518, 0.13649931,
         0.49496475]]),
 array([0.211, 0.2  , 0.087, 0.075, 0.065, 0.056, 0.056, 0.011, 0.007,
        0.007, 0.005, 0.004, 0.003, 0.003, 0.857, 0.832, 0.327, 0.207,
        0.117, 0.091, 0.083, 0.05 , 0.006, 0.006, 0.005, 0.004, 0.003,
        0.003, 0.003, 0.002, 0.001, 0.001, 0.001, 0.001, 0.934, 0.726,
        0.372, 0.268, 0.128, 0.061, 0.031, 0.021, 0.02 , 0.012, 0.006,
        0.005, 0.004, 0.004, 0.003, 0.002, 0.002, 0.00

In [1133]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = list(map(lambda x: torch.FloatTensor(x), train_test_split(X, y, test_size=0.1, random_state=0)))
X_train, y_train

(tensor([[0.2609, 1.0000, 0.8741,  ..., 0.6094, 0.4555, 0.5146],
         [0.2609, 0.9136, 0.8169,  ..., 0.6953, 0.2054, 0.4914],
         [0.1739, 0.6543, 0.8238,  ..., 0.7368, 0.1155, 0.4945],
         ...,
         [0.2174, 1.0000, 0.9222,  ..., 0.7230, 0.4038, 0.5025],
         [0.6957, 0.7778, 0.6636,  ..., 0.3324, 0.1553, 0.5121],
         [0.3478, 0.9877, 0.8719,  ..., 0.7368, 0.7602, 0.5468]]),
 tensor([0.0060, 0.0200, 0.0560, 0.2000, 0.0170, 0.6550, 0.5800, 0.8880, 0.1890,
         0.0450, 0.0930, 0.0200, 0.0260, 0.5500, 0.0020, 0.3710, 0.0010, 0.0050,
         0.0030, 0.0050, 0.9980, 0.3930, 0.0650, 0.0680, 0.0030, 0.0090, 0.0140,
         0.0560, 0.8390, 0.0020, 0.0110, 0.0480, 0.0010, 0.0830, 0.7570, 0.8570,
         0.3630, 0.7200, 0.9520, 0.0800, 0.9550, 0.0050, 0.6270, 0.0010, 0.0030,
         0.4870, 0.0010, 0.9610, 0.0040, 0.8790, 0.0200, 0.0020, 0.2700, 0.0070,
         0.0050, 0.0090, 0.0010, 0.5950, 0.0050, 0.1980, 0.0070, 0.0060, 0.0010,
         0.0730, 0.0010, 0.

In [1134]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Layers
        self.m1 = nn.Linear(X.shape[1], 256)
        self.f1 = nn.LeakyReLU()
        self.m2 = nn.Linear(256, 256)
        self.dropout = nn.Dropout(p=0.2)
        self.f2 = nn.LeakyReLU()
        self.m3 = nn.Linear(256, 64)
        self.f3 = nn.LeakyReLU()
        self.m4 = nn.Linear(64, 1)
        self.f4 = nn.ReLU()
    
    def forward(self, x):
        x = self.m1(x)
        x = self.f1(x)
        x = self.m2(x)
        x = self.dropout(x)
        x = self.f2(x)
        x = self.m3(x)
        x = self.f3(x)
        x = self.m4(x)
        x = self.f4(x)
        return x


In [1135]:
model = Network()

In [1136]:
# Hyperparameters
learning_rate = 3e-10
num_epochs = 15000

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.1, weight_decay=0.6)

# Training Model
train_loss = []
epoch_durs = []
for epoch in range(num_epochs):
  output = model(X_train)
  loss = torch.sum((output - y_train)**2)
  train_loss.append(loss.item())
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  print(f'epoch {epoch} of {num_epochs} loss: {loss.item()}\t\t\t\t\t\t\t\t', end='\r')

plt.plot(range(len(train_loss)), train_loss)
plt.show()

In [ ]:
output = model(X_val)
output = output.detach()
val_df = pd.DataFrame(np.hstack([output, y_val.reshape(-1, 1)]),
                      columns=['Pred', 'Actual'])

y_pred = model(X_val)
y_pred = y_pred.detach().reshape(1, -1)[0]
print(sklearn.metrics.r2_score(y_val, y_pred))
print(val_df.mean())
val_df

-0.12142064962922916
Pred      0.101069
Actual    0.207450
dtype: float32


,Pred,Actual
0,0.119319,0.028
1,0.107959,0.005
2,0.110929,0.074
3,0.089028,0.001
4,0.091712,0.157
5,0.078518,0.033
6,0.104533,0.813
7,0.094829,0.011
8,0.110322,0.228
9,0.106804,0.001


In [ ]:
cur_df = normalized_df.loc[normalized_df['Share'].isna()].drop(['Share'], axis=1)
cur_val_df = pd.DataFrame(columns=['Pred'], data=model(torch.FloatTensor(cur_df.drop(['Player'], axis=1).to_numpy())).detach())
cur_val_df['Player'] = normalized_df.loc[normalized_df['Share'].isna()]['Player']
cur_val_df = cur_val_df.sort_values(by=['Pred'])
cur_val_df = cur_val_df.dropna()
cur_val_df[-50:]

,Pred,Player
499,0.093451,Karl-Anthony Towns
368,0.093466,Mike Muscala
415,0.093687,Norman Powell
223,0.093690,Jrue Holiday
511,0.093912,Nikola Vučević
98,0.094011,Nic Claxton
238,0.094275,Kyrie Irving
182,0.094294,Draymond Green
305,0.094429,Kawhi Leonard
197,0.094438,Tim Hardaway Jr.


In [ ]:
def calc_pct(x):
    l = list(map(lambda i: i**100, x))
    return [val/sum(l) for val in l]
    
mvp_prediction_df = cur_val_df[-10:]
input = mvp_prediction_df['Pred'].to_numpy()
mvp_prediction_df['% Chance'] = calc_pct(input)
mvp_prediction_df = mvp_prediction_df.drop(['Pred'], axis=1)
mvp_prediction_df = mvp_prediction_df.reset_index(drop=True)
mvp_prediction_df = mvp_prediction_df.sort_values(by=['% Chance'], ascending=False)
mvp_prediction_df

/var/folders/_0/zqnz262926vbc83620bb5j9w0000gn/T/ipykernel_7842/2943891854.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mvp_prediction_df['% Chance'] = calc_pct(input)


,Player,% Chance
9,Jaden McDaniels,0.928316
8,DeAndre Jordan,0.052160
7,Myles Turner,0.018808
6,Terance Mann,0.000318
5,Lauri Markkanen,0.000165
4,T.J. McConnell,0.000069
3,Andrew Wiggins,0.000058
2,Kenrich Williams,0.000057
1,Scottie Barnes,0.000039
0,Mikal Bridges,0.000010
